In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
from quetzal.model import stepmodel
from quetzal.engine import engine, connectivity
from quetzal.io import excel

# Preparation of the transport network.
## Saves access and egress links for road network
## Needs all networks.


In [3]:
input_path = '../input_static/'
output_path = '../output/'
model_path = '../model/'

In [21]:
# Load scenario parameters
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

In [5]:
sm = stepmodel.read_json(model_path + 'de_pt_network_agg')
road = stepmodel.read_json(input_path + 'de_road_network')

In [6]:
len(sm.zones)

401

## Add access and egress links from zone centroids

In [7]:
# Compute controids
sm.centroids = sm.zones.copy()
sm.centroids['geometry'] = sm.centroids['geometry'].apply(lambda g: g.centroid)

In [23]:
# Compute road access and egress links
sm.zone_to_road = engine.ntlegs_from_centroids_and_nodes(
    sm.centroids,
    road.road_nodes,
    short_leg_speed=params['pt_access']['speed_walk'],
    long_leg_speed=params['pt_access']['speed_bicycle'],
    threshold=params['pt_access']['catchment_radius_walk'],
    n_neighbors=params['access-egress_links']['n_road'],
    coordinates_unit=sm.coordinates_unit)
sm.zone_to_road = sm.zone_to_road.loc[
    sm.zone_to_road['distance']<=params['access-egress_links']['road_max_dist']]
len(sm.zone_to_road.index)

802

In [10]:
# Compute road - PT links
# Don't connect to bus stations
sm.road_to_transit = engine.ntlegs_from_centroids_and_nodes(
    sm.nodes.loc[sm.nodes['route_type']!='bus'],
    road.road_nodes,
    short_leg_speed=params['pt_access']['speed_walk'],
    long_leg_speed=params['pt_access']['speed_bicycle'],
    threshold=params['pt_access']['catchment_radius_walk'],
    n_neighbors=params['access-egress_links']['n_road_pt'],
    coordinates_unit=sm.coordinates_unit)
sm.zone_to_road = sm.zone_to_road.loc[
    sm.zone_to_road['distance']<=params['access-egress_links']['road_pt_max_dist']]
len(sm.road_to_transit.index)

13762

### Parametrise access and egress links

In [27]:
# Road - PT connectors
sm.road_to_transit['distance'] = 0
sm.road_to_transit['time'] = params['access-egress_links']['time_road_pt']
sm.road_to_transit.sample(n=2)

,a,b,direction,distance,geometry,speed,time
2832,rs_DE600_29,122398,access,0,"LINESTRING (10.013313 53.5217, 10.013075 53.52...",5,300
6709,rs_DEG0I_8,35348608,access,0,"LINESTRING (11.169939 50.689453, 11.1622924 50...",5,300


In [28]:
# Road - centroid connectors
sm.zone_to_road['distance'] = 0
sm.zone_to_road['time'] = params['access-egress_links']['time_road']
sm.zone_to_road.sample(n=2)

,a,b,direction,distance,geometry,rank,speed_factor,short_leg_speed,long_leg_speed,speed,time
768,25230307,DEF04,eggress,0,LINESTRING (9.984259498570877 54.0812238270327...,0,1.042361,5,17,5.211805,300
600,60133018,DE11B,eggress,0,LINESTRING (9.725414426756322 49.5605175550927...,0,1.924182,5,17,9.620910,300


## Save model

In [29]:
# Drop unneccessary columns
cols = ['speed_factor', 'short_leg_speed', 'long_leg_speed', 'rank']
sm.zone_to_road.drop(cols, axis=1, inplace=True, errors='ignore')
sm.road_to_transit.drop(cols, axis=1, inplace=True, errors='ignore')

In [30]:
# Make tables lighter
cols = ['distance', 'speed', 'time']
sm.zone_to_road[cols] = sm.zone_to_road[cols].astype(int)
sm.road_to_transit[cols] = sm.road_to_transit[cols].astype(int)

In [32]:
sm.to_json(model_path + 'de_road_access_egress',
           only_attributes=['centroids', 'zone_to_road', 'road_to_transit'],
           encoding='utf-8')


to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████| 34/34 [00:02<00:00, 13.91it/s]
